# Triangle model
This interactive notebook runs a triangle model


## Run parameters 
This block is necessary for running with [papermill](https://papermill.readthedocs.io/en/latest/) in run_papermill.py.

In [ ]:
code_name = "hand_pretrain_40lr"
batch_name = None

# Model configs
ort_units = 119
pho_units = 250
sem_units = 2446
hidden_os_units = 500
hidden_op_units = 150
hidden_ps_units = 500
hidden_sp_units = 500
pho_cleanup_units = 50
sem_cleanup_units = 50
pho_noise_level = 0.0
sem_noise_level = 0.0
activation = "sigmoid"
tau = 1 / 3
max_unit_time = 4.0

# Training configs
pretrain_checkpoint = None
optimizer = "sgd"
learning_rate = 40.0
inject_error_ticks = 8
zero_error_radius = 0.1
loss_ramping = False

# Environment configs
wf_compression = "log"
wf_clip_low = 0
wf_clip_high = 10_000
task_names = ["ort_pho", "pho_pho", "triangle"]
tasks_ps = [0.2, 0.3, 0.5]
total_sample = 1_000_000
batch_size = 200

# Misc configs
rng_seed = 2021
save_freq = 10
which_gpu = 0


## System environment
Provision GPU resouses, set random seeds, and load environment variables

In [ ]:
import meta

meta.split_gpu(
    which_gpu=which_gpu
)  # IMPORTANT: do not import TensorFlow before this line

import os
import tensorflow as tf
import numpy as np
from dotenv import load_dotenv

# Set all seeds
os.environ["PYTHONHASHSEED"] = str(rng_seed)
tf.random.set_seed(rng_seed)
np.random.seed(rng_seed)

# Loads .env file
load_dotenv()


## Create run configuration

In [ ]:
# cfg = meta.Config.from_json(os.path.join(tf_root, "models", batch_name, code_name, "model_config.json"))   # Load from json
cfg = meta.Config.from_dict(**globals())
print(cfg)


## Create Experience
- `Experience()` defines what the model is trained on. It consists of one or more `Stage()`. 
- Each `Stage()` describes what tasks are the model trained with, and how often a task is used during training. It contains one or more `Task()`. 
- Each `Task()` contains how fast the corpus is opened (a set of word that can be sampled), defaults to full open.
- See the docstrings in each object for further details.
- CAUTION: if modified to multiple `Stage()`, Config() may not be storing everything needed to recreate Experience(). 

In [ ]:
from environment import Task, Stage, Experience

stages = [
    Stage(
        name="one",
        tasks=[Task(x) for x in cfg.task_names],
        stage_sample=cfg.total_sample,
        task_probability_start=cfg.tasks_ps,
    )
]

experience = Experience(stages)


## Create model trainer
- In tf.keras terminology, `Trainer()` is the compiled model. 
- It includes data, model, optimizer, metrics, and loss function, etc.
- Since each sub-task has its own states, trainer will create separate optimizer, metrics, losses in each task.


In [ ]:
from training import Trainer, TensorBoardManager

trainer = Trainer(cfg, experience)


## Train model
try_to_resume wiil restore from latest checkpoint if it exists. However, due to technical limit, Environment() will no longer be identical after resuming from checkpoint. 

In [ ]:
tb_manager = TensorBoardManager(
    cfg, trainer.model, trainer.train_metrics, trainer.train_losses
)
trainer.train(tensorboard_manager=tb_manager, try_to_resume=True)

del trainer  # Release memory before running tests

## Run tests
See benchmarks for a wide selection of tests.
TODO: relax shape constraints in TriangleModel()

In [ ]:
from meta import Config
import benchmarks
from modeling import TriangleModel
cfg = Config.from_json("models/triangle_pretrain_high_epsilon/triangle_pretrain_high_epsilon_r0001/model_config.json")
model = TriangleModel(cfg)
model.set_active_task("triangle")

In [ ]:

# Basic test
benchmarks.run_oral_eval(cfg, testset="train_r100")


# benchmarks.run_test1s(cfg.code_name, cfg.batch_name)  # Basic accuracy test only1

## All benchmarks
# benchmarks.main(cfg.code_name, cfg.batch_name)

## Full training set test
# import evaluate
# test = evaluate.Test(cfg)
# test.eval_train("triangle", to_bq=True)
